In [ ]:
---
title: 'Assignment 11'
author: "Rodrigue Beleho"
date: "October 21, 2018"
output: word_document
---

#Question 1
```{r}
df = read.csv ("C:\\Users\\student\\Desktop\\MATH 421\\adult.csv")
library(ggplot2)
library(caret)

colnames(df)[1] <- "age"
colnames(df)[2] <- "sector"
colnames(df)[3] <- "IDnumber"
colnames(df)[4] <- "education_level"
colnames(df)[5] <- "education_number"
colnames(df)[6] <- "marital_status"
colnames(df)[7] <- "profession"
colnames(df)[8] <- "family_background"
colnames(df)[9] <- "race"
colnames(df)[10] <- "gender"
colnames(df)[11] <- "capital_gain"
colnames(df)[12] <- "capital_loss"
colnames(df)[13] <- "weekly_hours"
colnames(df)[14] <- "country"
colnames(df)[15] <- "salary"

summary(df)
sum(is.na(df))
```

#Question 2 + Question 3
```{r}
for (i in 1:ncol(df)){
    df[df == "?"] = NA
    blanks <- sum(is.na(df[i])) 
    if (blanks >0){
      if(is.numeric(df[[i]]) == TRUE){
        df = preProcess(df, method = "medianImpute")
      }
  else {
      levels=unique(df[,i])
      df[,i][is.na(df[,i])]=levels[which.max(tabulate(match(df[,i], x=  levels)))]
    }
    }
}
```

#Question 4
```{r}

levels(df$profession) = c("BlueCJ", "Bluecollarjob", "MidblueCJ", "BlueCJ", "WHiteCJ", "BlueCJ", "BlueCJ", "MidblueCJ", "BlueCJ", "WhiteCJ", "WhiteCJ", "WhiteCJ", "WHiteCJ", "WhiteCJ", "BlueCJ")

levels(df$country) = c("North_A","Asia","North_A", "Asia","South_A", "South_A","Central_A", "South_A", "Central_A", "Europe", "Europe", "Europe", "Europe", "Central_A", "Central_A", "Europe", "Central_A", "Asia", "Europe", "Asia", "Asia", "Europe","Europe","Central_A","Asia", "Asia","Central_A", "Central_ A", "North_A","South_A","Asia","Europe","Europe","Central_A","Europe","Asia","Asia","Asia","Central_A","North_A","Asia","Europe")

levels(df$education_level) = c("HS_dropout", "HS_dropout", "HS_dropout", "HS_dropout","HS_dropout", "HS_dropout", "HS_dropout", "2year_College", "2year_College", "College_graduate", "SSJ_College", "HS_graduate" ,"SSJ_College", "HS_dropout","College_graduate", "2year_College")
              
```

#Question 5
```{r}
dummies_model <- dummyVars(salary ~., data=df)
trainData_mat <- predict(dummies_model, newdata = df)

trainData <- data.frame(trainData_mat)
trainData$salary <- df$salary
```

#Question 6
```{r}
trainData2 <- preProcess(trainData, method = c("center", "scale"))
trainData2
```

#Question 7
```{r}
names(df)[15] = "target"
set.seed(2018)
splitIndex <- createDataPartition(df$target, p =.70, list= FALSE, times=1)
train <- df[splitIndex,]
test <- df[-splitIndex,]

library(rpart)
mytree <- rpart(target ~ ., data = train, method = "class")

pred <- predict(mytree, test, type = "class")
cm = confusionMatrix( data = pred, reference = test$target, positive= " >50K")

print(cm)
```


#Question 8
```{r}

library(ranger)
models = ranger(target ~., data = train)
pred = predict(models, data = test)$predictions
cm = confusionMatrix(pred, test$target, positive= " >50K")
cm[[3]][1]

cm[[4]][11]
```


#Question 9
```{r}

till9 = function(df){
  for (j in 1: ncol(df)){
    if(is.numeric(df[,j])){
      df[,j][is.na(df[,j])] <- mean(df[,j], na.rm=TRUE)
    }
  }
  return(df)
}
df$target <- as.numeric(df$target)
df = till9(df)

library(caret)

levels(df$profession) = c("BlueCJ", "Bluecollarjob", "MidblueCJ", "BlueCJ", "WHiteCJ", "BlueCJ", "BlueCJ", "MidblueCJ", "BlueCJ", "WhiteCJ", "WhiteCJ", "WhiteCJ", "WHiteCJ", "WhiteCJ", "BlueCJ")

levels(df$country) = c("North_A","Asia","North_A", "Asia","South_A", "South_A","Central_A", "South_A", "Central_A", "Europe", "Europe", "Europe", "Europe", "Central_A", "Central_A", "Europe", "Central_A", "Asia", "Europe", "Asia", "Asia", "Europe","Europe","Central_A","Asia", "Asia","Central_A", "Central_ A", "North_A","South_A","Asia","Europe","Europe","Central_A","Europe","Asia","Asia","Asia","Central_A","North_A","Asia","Europe")

levels(df$education_level) = c("HS_dropout", "HS_dropout", "HS_dropout", "HS_dropout","HS_dropout", "HS_dropout", "HS_dropout", "2year_College", "2year_College", "College_graduate", "SSJ_College", "HS_graduate" ,"SSJ_College", "HS_dropout","College_graduate", "2year_College")

dummies_model2 <- dummyVars(~., data=df)
adultData2 <- data.frame(predict(dummies_model2, newdata = df))

adultData2 <- data.frame(adultData2)
adultData2$target[adultData2$target ==1] <- " <=50K"
adultData2$target[adultData2$target ==2] <- " >50K"

preProcess_missingdata_model <- preProcess(adultData2, method = c("center", "scale"))
trainData2 <- predict(preProcess_missingdata_model, newdata = adultData2)

library(lattice)
library(caret)
#names(trainData2)[50] = "target"
set.seed(2018)
splitIndex <- createDataPartition(trainData2$target, p =.70, list = FALSE, times = 1)

train2 <- trainData2[splitIndex,]
test2 <- trainData2 [-splitIndex,]

library(rpart)
tree2 <- rpart(target ~ ., data = train2, method = "class")
library(rattle)
fancyRpartPlot(tree2)

predictadult2 <- predict(tree2, test2, type = "class")
cm1 = confusionMatrix (table (data= predictadult2, reference = test2$target))
cm1
```




#Question 10
```{r}

library(lattice)
library(ggplot2)
library(caret)

preProcess_missindata_model <- preProcess(df, method = "knnImpute")
adultData3 <- predict(preProcess_missindata_model, newdata = df)

levels(df$profession) = c("BlueCJ", "Bluecollarjob", "MidblueCJ", "BlueCJ", "WHiteCJ", "BlueCJ", "BlueCJ", "MidblueCJ", "BlueCJ", "WhiteCJ", "WhiteCJ", "WhiteCJ", "WHiteCJ", "WhiteCJ", "BlueCJ")

levels(df$country) = c("North_A","Asia","North_A", "Asia","South_A", "South_A","Central_A", "South_A", "Central_A", "Europe", "Europe", "Europe", "Europe", "Central_A", "Central_A", "Europe", "Central_A", "Asia", "Europe", "Asia", "Asia", "Europe","Europe","Central_A","Asia", "Asia","Central_A", "Central_ A", "North_A","South_A","Asia","Europe","Europe","Central_A","Europe","Asia","Asia","Asia","Central_A","North_A","Asia","Europe")

levels(df$education_level) = c("HS_dropout", "HS_dropout", "HS_dropout", "HS_dropout","HS_dropout", "HS_dropout", "HS_dropout", "2year_College", "2year_College", "College_graduate", "SSJ_College", "HS_graduate" ,"SSJ_College", "HS_dropout","College_graduate", "2year_College")


df$target <- as.numeric(df$target)
dummies_model2 <- dummyVars(~., data=df)
adultData3 <- data.frame(predict(dummies_model2, newdata = df))


adultData3 <- data.frame(adultData3)
adultData3$target[adultData3$target ==1] <- " <=50K"
adultData3$target[adultData3$target ==2] <- " >50K"

library(lattice)
library(caret)

splitIndex <- createDataPartition(adultData3$target, p = .70, list = FALSE, times = 1)
set.seed(2018)
train3 <- adultData3 [splitIndex,]
test3 <- adultData3 [-splitIndex,]

library(rpart)
tree3 <- rpart(target ~ ., data = train3, method = "class")
library(rattle)
fancyRpartPlot(tree3)
```


#Question 11
```{r}

trainData4 <- preProcess(df, method = c("center", "scale"))
adultData4 <- predict(trainData4, newdata = df)

levels(df$profession) = c("BlueCJ", "Bluecollarjob", "MidblueCJ", "BlueCJ", "WHiteCJ", "BlueCJ", "BlueCJ", "MidblueCJ", "BlueCJ", "WhiteCJ", "WhiteCJ", "WhiteCJ", "WHiteCJ", "WhiteCJ", "BlueCJ")

levels(df$country) = c("North_A","Asia","North_A", "Asia","South_A", "South_A","Central_A", "South_A", "Central_A", "Europe", "Europe", "Europe", "Europe", "Central_A", "Central_A", "Europe", "Central_A", "Asia", "Europe", "Asia", "Asia", "Europe","Europe","Central_A","Asia", "Asia","Central_A", "Central_ A", "North_A","South_A","Asia","Europe","Europe","Central_A","Europe","Asia","Asia","Asia","Central_A","North_A","Asia","Europe")

levels(df$education_level) = c("HS_dropout", "HS_dropout", "HS_dropout", "HS_dropout","HS_dropout", "HS_dropout", "HS_dropout", "2year_College", "2year_College", "College_graduate", "SSJ_College", "HS_graduate" ,"SSJ_College", "HS_dropout","College_graduate", "2year_College")


df$target <- as.numeric(df$target)
dummies_model4 <- dummyVars(~., data=df)
adultData4 <- data.frame(predict(dummies_model4, newdata = df))


adultData4 <- data.frame(adultData4)
adultData4$target[adultData4$target ==1] <- " <=50K"
adultData4$target[adultData4$target ==2] <- " >50K"


splitIndex <- createDataPartition(adultData4$target, p = .70, list = FALSE, times = 1)
set.seed(2018)
train4 <- adultData4 [splitIndex,]
test4 <- adultData4 [-splitIndex,]

library(rpart)
tree4 <- rpart(target ~ ., data = train4, method = "class")
library(rattle)
fancyRpartPlot(tree4)

```


#Question 12
```{r}
adultData5 <- df

levels(adultData5$profession) = c("BlueCJ", "Bluecollarjob", "MidblueCJ", "BlueCJ", "WHiteCJ", "BlueCJ", "BlueCJ", "MidblueCJ", "BlueCJ", "WhiteCJ", "WhiteCJ", "WhiteCJ", "WHiteCJ", "WhiteCJ", "BlueCJ")

levels(adultData5$country) = c("North_A","Asia","North_A", "Asia","South_A", "South_A","Central_A", "South_A", "Central_A", "Europe", "Europe", "Europe", "Europe", "Central_A", "Central_A", "Europe", "Central_A", "Asia", "Europe", "Asia", "Asia", "Europe","Europe","Central_A","Asia", "Asia","Central_A", "Central_ A", "North_A","South_A","Asia","Europe","Europe","Central_A","Europe","Asia","Asia","Asia","Central_A","North_A","Asia","Europe")

levels(adultData5$education_level) = c("HS_dropout", "HS_dropout", "HS_dropout", "HS_dropout","HS_dropout", "HS_dropout", "HS_dropout", "2year_College", "2year_College", "College_graduate", "SSJ_College", "HS_graduate" ,"SSJ_College", "HS_dropout","College_graduate", "2year_College")

 for (i in 1: ncol(adultData5)){
    if(is.factor(adultData5[,i])){
      df[,i] = as.numeric(adultData5[,i])
    }
  }


df$target <- as.numeric(df$target)
dummies_model5 <- dummyVars(~., data=df)
adultData5 <- data.frame(predict(dummies_model5, newdata = df))


adultData5 <- data.frame(adultData5)
adultData5$target[adultData5$target ==1] <- " <=50K"
adultData5$target[adultData5$target ==2] <- " >50K"


splitIndex <- createDataPartition(adultData5$target, p = .70, list = FALSE, times = 1)
set.seed(2018)
train5 <- adultData5 [splitIndex,]
test5 <- adultData5 [-splitIndex,]

library(rpart)
tree5 <- rpart(target ~ ., data = train5, method = "class")
library(rattle)
fancyRpartPlot(tree5)

```


#Question 13
```{r}
adultData6 <- df

df$target <- as.numeric(df$target)
dummies_model6 <- dummyVars(~., data=df)
adultData6 <- data.frame(predict(dummies_model6, newdata = df))


adultData6 <- data.frame(adultData6)
adultData6$target[adultData6$target ==1] <- " <=50K"
adultData6$target[adultData6$target ==2] <- " >50K"

trainData6 <- preProcess(df, method = c("center", "scale"))
adultData6 <- predict(trainData6, newdata = df)


splitIndex <- createDataPartition(adultData5$target, p = .70, list = FALSE, times = 1)
set.seed(2018)
train6 <- adultData6 [splitIndex,]
test6 <- adultData6 [-splitIndex,]

library(rpart)
tree6 <- rpart(target ~ ., data = train5, method = "class")
library(rattle)
fancyRpartPlot(tree6)

```